In [12]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pathresults = "../../results/"
#pathresults = "/DUMUXDune27/DUMUX/dumux-rosi/python/paperSc/results/"
data_file_delimiter = ","
import plotnine

from plotnine import *

# get param set
paramsets =  pd.read_csv('../../output_random_rows.csv')
paramsets = paramsets.drop(paramsets.columns.tolist()[-1],axis=1)
outputvalsname =  ['wat','cs']
column_names = outputvalsname + paramsets.columns.tolist()
paramsets.drop('theta',inplace=True, axis=1)

df = pd.read_pickle("./pset99_allcell.pkl")  # cid, pSet, rr, 1 or 9


nKonz = 2
volIdx=3
lenIdx=4
stIdx=5
orgIdId=6
relLenId=7
coordId = 8

nToGet = coordId#elements + 3d coordinates of y , node+vol+len+st 


In [13]:
df.shape

(9, 99)

In [14]:
konzcs = df.loc[1]# pSet, rr, 9
vols = df.loc[volIdx]# pSet, rr, 9
lens = df.loc[lenIdx]# pSet, rr, 1
orgIds = df.loc[orgIdId]# pSet, rr, 1
reLens =  df.loc[relLenId]
coords =  df.loc[coordId]

In [22]:
reLens[0][1]

2.0

In [52]:
coords[:][19]

array([[0.05499005, 0.06596616, 0.07913313, ..., 0.16387281, 0.19658214,
        0.2358203 ],
       [0.02202224, 0.02647568, 0.03182971, ..., 0.06649286, 0.07993935,
        0.09610504],
       [0.02237719, 0.02769669, 0.03428072, ..., 0.08045234, 0.09957742,
        0.1232489 ],
       ...,
       [0.02251649, 0.02818273, 0.03527487, ..., 0.08657551, 0.10836212,
        0.13563129],
       [0.02251649, 0.02818273, 0.03527487, ..., 0.08657551, 0.10836212,
        0.13563129],
       [0.02394735, 0.03340021, 0.04658443, ..., 0.17628183, 0.24586642,
        0.34291849]])

In [18]:
repr([sum(orgIds.loc[pSet]==1) for pSet in range(99) ])

'[20, 20, 20, 20, 21, 20, 20, 21, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 21, 20, 20, 20, 21, 20, 20, 20, 20, 20, 20, 20, 20, 21, 20, 21, 20, 20, 20, 20, 20, 20, 20, 21, 20, 21, 21, 20, 20, 20, 20, 20, 20, 20, 20, 20, 21, 20, 21, 21, 20, 21, 20, 20, 20, 21, 21, 21, 20, 20, 21, 20]'

In [42]:
dfnew = pd.DataFrame(columns=['locCylId','coords','konzcs','vols','lens','reLens','pSet'])

for pSet in range(99):
    
    cyltr = sum(orgIds.loc[pSet]==1)# 21 or 22
    locCylId = np.array([np.full(9,ii) for ii in range(cyltr)])
    locCellId = np.full((cyltr,9),np.array([i for i in range(9)]))
    
    myData = pd.DataFrame({'locCylId':locCylId.reshape(-1)})
    
    myData['locCellId']= locCellId.reshape(-1)
    myData['coords']= coords.loc[pSet][orgIds.loc[pSet]==1].reshape(-1)
    myData['konzcs']= konzcs.loc[pSet][orgIds.loc[pSet]==1].reshape(-1)
    myData['vols']= vols.loc[pSet][orgIds.loc[pSet]==1].reshape(-1)
    
    lens__ = lens.loc[pSet][orgIds.loc[pSet]==1]
    lens_ = np.array([np.full(9, ll) for ll in lens__])
    reLens__ = reLens.loc[pSet][orgIds.loc[pSet]==1]
    reLens_ = np.array([np.full(9, ll) for ll in reLens__])
    myData['lens']= lens_.reshape(-1)
    myData['reLens']= reLens_.reshape(-1)
    myData['pSet'] = pSet
    
    dfnew = pd.concat([dfnew, myData], ignore_index=True)
    

In [47]:
print(dfnew.loc[dfnew['locCellId']==0,'coords'].max(),
      dfnew.loc[dfnew['locCellId']==6,'coords'].min())#0.06cm?

0.057472616323468714 0.06649286282483836


In [39]:
obsDis = 0.06

dfnew = pd.DataFrame(columns=['locCylId','cs','reLens','pSet'])

for pSet in range(99):
    if len(coords.loc[pSet]) > 0:
    
        cyltr = sum(orgIds.loc[pSet]==1)# 19-22
        locCylId = np.array([ii for ii in range(cyltr)])

        myData = pd.DataFrame({'locCylId':locCylId.reshape(-1)})

        #myData['locCellId']= locCellId#.reshape(-1)
        #myData['coords']= coords.loc[pSet]#.reshape(-1)

        #myData['konzcs']= konzcs.loc[pSet]#.reshape(-1)
        #myData['vols']= vols.loc[pSet]#.reshape(-1)

        #myData['lens']= lens.loc[pSet]
        myData['reLens']= reLens[pSet][orgIds.loc[pSet]==1]
        myData['pSet'] =  pSet

        y_interp =np.array([np.interp(obsDis, coords.loc[pSet][cylloc],
                                      konzcs[pSet][cylloc]
                                     ) for cylloc in range(cyltr)])
        myData['konzc'] = y_interp
        dfnew = pd.concat([dfnew, myData], ignore_index=True)

In [40]:
dfnew

,locCylId,cs,reLens,pSet,konzc
0,0,NaN,1.0,0,9.185186e-06
1,1,NaN,2.0,0,1.960936e-06
2,2,NaN,3.0,0,2.179401e-06
3,3,NaN,4.0,0,2.135895e-06
4,4,NaN,5.0,0,2.129707e-06
...,...,...,...,...,...
1992,15,NaN,16.0,98,6.567472e-07
1993,16,NaN,17.0,98,8.728945e-07
1994,17,NaN,18.0,98,9.250490e-07
1995,18,NaN,19.0,98,1.501308e-05


In [20]:
dfnew['rIn'] = -1
dfnew['rOut'] = -1
dfnew.loc[dfnew['locCellId']==0,'rIn'] = 0.05
for pSet in range(99):
    cyltr = sum(orgIds.loc[pSet]==1)
    for cylid in range(cyltr):
        #lenCyl = dfnew[dfnew['pSet']==pSet][dfnew['locCylId']==cylid]['rIn'] 
        for cellid in range(1,9):
            rowcell = dfnew.loc[(dfnew['pSet']==pSet)&(
                dfnew['locCylId']==cylid)&(
                dfnew['locCellId']==cellid)].index.tolist()[0]
            lencyl = dfnew.at[rowcell,'lens']
            vol =  dfnew.at[rowcell,'vols']
            dfnew.at[rowcell,'rOut'] = np.sqrt(vol/(np.pi*lencyl) +dfnew.at[rowcell,'rIn']**2 )
            
            if cellid < 8:
                rownextcell = dfnew.loc[(dfnew['pSet']==pSet)&(dfnew['locCylId']==cylid)&(dfnew['locCellId']==0)].index.tolist()[0]
                dfnew.at[rownextcell,'rIn'] = dfnew.at[rowcell,'rOut']


In [24]:
dfnew.head(n=10)#[dfnew['coords'] < 0.05]

,locCylId,coords,konzcs,vols,lens,reLens,pSet,locCellId,rIn,rOut
0,0,0.054990,1.073521e-05,0.003448,1.0,1.0,0,0.0,1.006989,-1.000000
1,0,0.065966,7.339324e-06,0.004962,1.0,1.0,0,1.0,-1.000000,1.000789
2,0,0.079133,4.557653e-06,0.007141,1.0,1.0,0,2.0,-1.000000,1.001136
3,0,0.094928,2.556139e-06,0.010276,1.0,1.0,0,3.0,-1.000000,1.001634
4,0,0.113876,1.326372e-06,0.014788,1.0,1.0,0,4.0,-1.000000,1.002351
5,0,0.136606,7.028279e-07,0.021280,1.0,1.0,0,5.0,-1.000000,1.003381
6,0,0.163873,4.505152e-07,0.030623,1.0,1.0,0,6.0,-1.000000,1.004862
7,0,0.196582,3.717496e-07,0.044068,1.0,1.0,0,7.0,-1.000000,1.006989
8,0,0.235820,3.537835e-07,0.063415,1.0,1.0,0,8.0,-1.000000,1.010042
9,1,0.022022,8.987728e-06,0.000560,1.0,2.0,0,0.0,1.001173,-1.000000


In [72]:
dfnew.loc[dfnew['locCellId']==0,'coords']# = 0.05

0        0.054990
9        0.022022
18       0.022174
27       0.022305
36       0.022305
           ...   
18675    0.023947
18684    0.023947
18693    0.057473
18702    0.022816
18711    0.023422
Name: coords, Length: 2080, dtype: float64

In [74]:
dfnew.to_pickle('./cyl10to11_celss2.pkl')

In [77]:
print(dfnew.loc[dfnew['locCellId']==0,'coords'].min(),dfnew.loc[dfnew['locCellId']==0,'coords'].max())
print(dfnew.loc[dfnew['locCellId']==8,'coords'].min(),dfnew.loc[dfnew['locCellId']==8,'coords'].max())

0.02202224498672597 0.057472616323468714
0.09610504338489201 0.47702422259471233


In [ ]:
dist = 0.0

In [ ]:
dfnew.loc[dfnew['locCellId']==0,'rIn']

In [ ]:
plt.scatter(dfnew['yX'],df['yZ'] , #np.log(df['cc']),np.log(df['co']), 
            c= np.log(df['cs']),#s=df['cs'],
            alpha=0.3, edgecolors ='none')
plt.rcParams.update({'figure.figsize':(5,5), 'figure.dpi':100})
#plt.title()
plt.colorbar()
plt.xlabel('cs')
plt.ylabel('co +cc')
plt.show()

In [41]:

cyltr = sum(orgIds.loc[pSet]==1)# 21
locCylId = np.array([np.full(9,ii) for ii in range(cyltr)])
locCellId = np.full((21,cyltr),np.array([i for i in range(cyltr)]))

myData = pd.DataFrame({'locCylId':locCylId.reshape(-1)})

#myData['locCellId']= locCellId.reshape(-1)

In [49]:
locCylId.shape

(22, 9)

In [50]:
locCellId.shape

(21, 9)

In [33]:
lens__ = lens.loc[pSet][orgIds.loc[0]==1]

In [36]:
lens_ = np.array([np.full(9, ll) for ll in lens__])

In [37]:
lens_

array([[1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.]])

In [21]:
cc.shape

(21, 9)

In [26]:
cc[0]

array([0.05499005, 0.06596616, 0.07913313, 0.09492826, 0.11387611,
       0.136606  , 0.16387281, 0.19658214, 0.2358203 ])

In [25]:
cc.reshape(-1)

array([0.05499005, 0.06596616, 0.07913313, 0.09492826, 0.11387611,
       0.136606  , 0.16387281, 0.19658214, 0.2358203 , 0.02202224,
       0.02647568, 0.03182971, 0.03826646, 0.04600488, 0.05530819,
       0.06649286, 0.07993935, 0.09610504, 0.02217415, 0.02699516,
       0.03286432, 0.04000953, 0.04870822, 0.05929814, 0.07219047,
       0.08788579, 0.10699352, 0.02230469, 0.02744521, 0.03377047,
       0.04155351, 0.05113028, 0.0629142 , 0.07741395, 0.09525543,
       0.11720881, 0.02230469, 0.02744521, 0.03377047, 0.04155351,
       0.05113028, 0.0629142 , 0.07741395, 0.09525543, 0.11720881,
       0.02202224, 0.02647568, 0.03182971, 0.03826646, 0.04600488,
       0.05530819, 0.06649286, 0.07993935, 0.09610504, 0.0227442 ,
       0.02898565, 0.03693989, 0.04707692, 0.05999575, 0.07645977,
       0.09744183, 0.12418179, 0.15825972, 0.02363763, 0.03223614,
       0.04396248, 0.05995442, 0.08176365, 0.11150628, 0.15206819,
       0.20738504, 0.28282415, 0.0227442 , 0.02898565, 0.03693